In [1]:
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb

folder = 'C:\\Users\\nemrodov\\Documents\\Ilya_study\\Analysis'
import numpy as np
import pandas as pd
perc_ind=['02','03','04','05','06','07','08','09','10','11','12','13','14','15','17']
imag_ind=['02','03','04','05','06','07','08','09','10','11','12','13','14','15','17']
perc_file=[folder+'/conf_mat_perc_'+ i +'.csv' for i in perc_ind]
imag_file=[folder+'/conf_IR_'+ i +'_imag.csv' for i in perc_ind]
data = load_multiple_conf_mats(folder, perc_file, imag_file)
perc_list = data[0]
imag_list = data[1]

In [2]:
subject = 13
mat = isolate_perc_as_imag(perc_list[subject], imag_list[subject])

In [5]:
mat[1]

,est,mcy,sjo,sgo,tsw
est,0.000000,0.625000,0.666667,0.625000,0.666667
mcy,0.625000,0.000000,0.666667,0.541667,0.458333
sjo,0.666667,0.666667,0.000000,0.708333,0.875000
sgo,0.625000,0.541667,0.708333,0.000000,0.583333
tsw,0.666667,0.458333,0.875000,0.583333,0.000000


In [ ]:
from scipy.spatial.distance import pdist, squareform
import numpy as np
acc_perc=[]
acc_imag=[]
for i in range(len(perc_list)):
    out = inter_sub_corr(perc_list[i], imag_list[i])
    mat = out[0]
    acc_perc.append(np.mean(squareform(mat.iloc[:5,:5])))
    mat = out[1]
    acc_imag.append(np.mean(squareform(mat)))
acc_perc = np.array(acc_perc)
acc_imag = np.array(acc_imag)

In [ ]:
mat

In [ ]:
from scipy.spatial.distance import squareform
perc_up_arr =  np.array([squareform(i.values[:50,:50]) for i in perc_list])
perc_in_arr =  np.array([squareform(i.values[50:,50:]) for i in perc_list])
perc_up_fam_arr =  np.array([squareform(i.values[25:50,25:50]) for i in perc_list])
perc_up_unf_arr =  np.array([squareform(i.values[:25,:25]) for i in perc_list])
perc_in_fam_arr =  np.array([squareform(i.values[75:,75:]) for i in perc_list])
perc_in_unf_arr =  np.array([squareform(i.values[50:75,50:75]) for i in perc_list])
imag_arr =  np.array([squareform(i.values) for i in imag_list])

In [ ]:
perc_up_corr = pd.DataFrame(np.corrcoef(perc_up_arr))
perc_in_corr = pd.DataFrame(np.corrcoef(perc_in_arr))
perc_up_unf_corr = pd.DataFrame(np.corrcoef(perc_up_unf_arr))
perc_up_fam_corr = pd.DataFrame(np.corrcoef(perc_up_fam_arr))
perc_in_unf_corr = pd.DataFrame(np.corrcoef(perc_in_unf_arr ))
perc_in_fam_corr = pd.DataFrame(np.corrcoef(perc_in_fam_arr))
imag_corr = pd.DataFrame(np.corrcoef(imag_arr))

In [ ]:
perc_up_corr

In [ ]:
perc_in_corr

In [ ]:
imag_corr

In [ ]:
perc_up_unf_corr

In [ ]:
perc_up_fam_corr

In [ ]:
perc_in_unf_corr

In [ ]:
perc_in_fam_corr

In [ ]:
len(perc_list)

In [ ]:
len(imag_list)

In [ ]:
print(perc_list[0])

# Figures

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import cm
color=cm.rainbow(np.linspace(0,1,14))
fig, ax = plt.subplots(1,1,figsize=(8,6), dpi= 80)
x = np.arange(2)
ax = sns.barplot(x=x, y=[np.mean(acc_perc),np.mean(acc_imag)], linewidth=2.5,edgecolor=['grey','grey'],
                 facecolor=(0.5, 0.5, 0.5, 0.5))
#plt.bar(x, [np.mean(acc_perc),np.mean(acc_imag)])
both = list(zip(acc_perc,acc_imag))
for i in range(len(both)):
    sns.pointplot(x=x, y=both[i],kind="point",dodge=False,ax=ax,linestyles = '--',color=color[i])
plt.ylim((0.45, 0.8))
plt.xticks(x, ('perception', 'imagery'),fontsize=20)
plt.axhline(y=0.5, color='black', linestyle='-')
plt.ylabel('Accuracy [%]', fontsize=20)
fig.savefig('slope.png', dpi=300)

In [ ]:
from scipy.spatial.distance import pdist, squareform
import numpy as np
from sklearn.manifold import MDS
from scipy.spatial import procrustes
aver_perc=[]
aver_imag=[]
for i in range(len(perc_list)):
    out = inter_sub_corr(perc_list[i], imag_list[i])
    mat=out[0]
    aver_perc.append(mat.iloc[:5,:5].values)
    mat = out[1]
    aver_imag.append(mat.values)
aver_perc = np.array(aver_perc)
aver_imag = np.array(aver_imag)
aver_perc = np.mean(aver_perc,axis=0)
aver_imag = np.mean(aver_imag,axis=0)
embedding = MDS(n_components=2)
transf_perc = embedding.fit_transform(aver_perc)
transf_imag = embedding.fit_transform(aver_imag)
transf_perc, transf_imag, disparity = procrustes(transf_perc, transf_imag)

In [ ]:
transf_perc.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import cm
fig, ax = plt.subplots(1,1,figsize=(8,8), dpi= 80)
sns.scatterplot(x=transf_perc[:,0], y=transf_perc[:,1],color='red')
sns.scatterplot(x=transf_imag[:,0], y=transf_imag[:,1],color='blue')
ax.legend(['perception', 'imagery'], fontsize=18, frameon = False)
ax.set_xlabel('Dimension 1', fontsize = 20)
ax.set_ylabel('Dimension 2', fontsize = 20)
for i in range(transf_imag.shape[0]):
    sns.lineplot(x=[transf_perc[i,0],transf_imag[i,0]],y=[transf_perc[i,1],transf_imag[i,1]],color='grey')
fig.savefig('spaces.png', dpi=300)

In [ ]:
transf_perc

In [ ]:
disparity


In [ ]:
from scipy import stats
print(acc_perc.shape)
stats.pearsonr(acc_perc, acc_imag)

In [ ]:
from scipy import stats
print(squareform(aver_perc).shape)
stats.pearsonr(squareform(aver_perc),squareform(aver_imag))

In [ ]:
acc_perc.shape

## Accuracy for perceptual corresponding to imagery and imagery faces

In [18]:
%run EEG_auxiliary_module.ipynb
%run general_tools.ipynb
from scipy.spatial.distance import pdist, squareform
import numpy as np
import pandas as pd


folder = 'C:\\Users\\nemrodov\\Documents\\Ilya_study\\Analysis'

perc_ind=['02','03','04','05','06','07','08','09','10','11','12','13','14','15','17']
imag_ind=['02','03','04','05','06','07','08','09','10','11','12','13','14','15','17']
perc_file=[folder+'/conf_mat_perc_'+ i +'.csv' for i in perc_ind]
imag_file=[folder+'/conf_IR_'+ i +'_imag.csv' for i in perc_ind]
data = load_multiple_conf_mats(folder, perc_file, imag_file)
perc_list = data[0]
imag_list = data[1]
perc_up_acc = []
perc_in_acc = []
imag_acc = []
for i in range(len(perc_ind)):
    mat = isolate_perc_as_imag(perc_list[i], imag_list[i])
    imag_acc.append(np.mean(squareform(mat[1])))
    perc_up_acc.append(np.mean(squareform(mat[0].iloc[:5,:5])))
    perc_in_acc.append(np.mean(squareform(mat[0].iloc[-5:,-5:])))

In [19]:
perc_in_acc

[0.459090909090909,
 0.5083333333333333,
 0.5541666666666666,
 0.4624999999999999,
 0.575,
 0.5791666666666666,
 0.5875,
 0.5458333333333333,
 0.4999999999999999,
 0.6916666666666665,
 0.5666666666666667,
 0.5333333333333333,
 0.5708333333333333,
 0.7,
 0.675]